# DAMN - Decentralized AI Memory Network
## Multi-Agent Knowledge Sharing Demo

This notebook demonstrates how autonomous agents (UAVs) can share learned experiences using blockchain + IPFS.

**Prerequisites:**
- Ethereum wallet with Sepolia ETH ([Get free testnet ETH](https://sepoliafaucet.com))
- Pinata IPFS account ([Sign up free](https://pinata.cloud))
- RPC endpoint (Infura/Alchemy/RockX)

**Setup:**
1. Copy `.env.example` to `.env`
2. Fill in your credentials
3. Run cells in order

In [ ]:
# Cell 1: Install Dependencies
!pip install -q web3 python-dotenv requests
print("✅ Dependencies installed!")

In [ ]:
# Cell 2: Load Environment Variables
import os
from dotenv import load_dotenv

load_dotenv()  # Load from .env file

# Configuration from environment
PINATA_API_KEY = os.getenv("PINATA_API_KEY")
PINATA_API_SECRET = os.getenv("PINATA_API_SECRET")
PRIVATE_KEY = os.getenv("PRIVATE_KEY")
SEPOLIA_RPC = os.getenv("SEPOLIA_RPC")
CONTRACT_ADDRESS = "0xacAABF9A47d1Df7f2f698ad9033da10CD374B8c4"

# Validate configuration
if not all([PINATA_API_KEY, PINATA_API_SECRET, PRIVATE_KEY, SEPOLIA_RPC]):
    raise ValueError("❌ Missing environment variables! Check your .env file")

print("✅ Configuration loaded!")
print(f"📍 Contract: {CONTRACT_ADDRESS}")

In [ ]:
# Cell 3: Pinata IPFS Helper Class
import requests
import json

class PinataIPFS:
    """Helper class for Pinata IPFS operations"""
    
    def __init__(self, api_key, api_secret):
        self.api_key = api_key
        self.api_secret = api_secret
        self.base_url = "https://api.pinata.cloud"
        self.headers = {
            'pinata_api_key': api_key,
            'pinata_secret_api_key': api_secret,
            'Content-Type': 'application/json'
        }
    
    def pin_json(self, json_data, name="memory"):
        """Upload JSON to IPFS"""
        url = f"{self.base_url}/pinning/pinJSONToIPFS"
        payload = {
            "pinataContent": json_data,
            "pinataMetadata": {"name": name}
        }
        response = requests.post(url, json=payload, headers=self.headers)
        return response.json()
    
    def get_json(self, ipfs_hash):
        """Retrieve JSON from IPFS"""
        url = f"https://gateway.pinata.cloud/ipfs/{ipfs_hash}"
        response = requests.get(url)
        return response.json()

# Initialize Pinata
pinata = PinataIPFS(PINATA_API_KEY, PINATA_API_SECRET)
print("✅ Pinata helper created!")

In [ ]:
# Cell 4: Test IPFS Storage
from datetime import datetime

# Create test memory
test_memory = {
    "agent_id": "UAV-001",
    "task": "obstacle_navigation",
    "location": {"lat": 28.6139, "lon": 77.2090, "alt": 150},
    "obstacle_detected": True,
    "obstacle_type": "building",
    "safe_maneuver": "turn_left_30_degrees",
    "success_rate": 0.95,
    "timestamp": datetime.now().isoformat()
}

# Upload to IPFS
print("📤 Uploading memory to IPFS...")
result = pinata.pin_json(test_memory, name="DAMN_Test_Memory")

if 'IpfsHash' in result:
    ipfs_hash = result['IpfsHash']
    print(f"\n✅ SUCCESS! Memory stored on IPFS")
    print(f"📦 IPFS Hash (CID): {ipfs_hash}")
    print(f"🌐 View at: https://gateway.pinata.cloud/ipfs/{ipfs_hash}")
    
    # Test retrieval
    print("\n📥 Testing memory retrieval...")
    retrieved = pinata.get_json(ipfs_hash)
    print(f"✅ Retrieved: {retrieved['task']} by {retrieved['agent_id']}")
else:
    print(f"❌ Error: {result}")

In [ ]:
# Cell 5: Setup Blockchain Connection
from web3 import Web3

# Connect to Sepolia
w3 = Web3(Web3.HTTPProvider(SEPOLIA_RPC))

if w3.is_connected():
    print("✅ Connected to Sepolia testnet!")
    print(f"📊 Current block: {w3.eth.block_number}")
    print(f"⛽ Current gas price: {w3.from_wei(w3.eth.gas_price, 'gwei'):.4f} gwei")
else:
    raise Exception("❌ Cannot connect to Sepolia")

# Contract ABI
CONTRACT_ABI = [
    {
        "inputs": [
            {"internalType": "string", "name": "_memoryId", "type": "string"},
            {"internalType": "string", "name": "_ipfsCID", "type": "string"},
            {"internalType": "string", "name": "_agentId", "type": "string"},
            {"internalType": "string", "name": "_taskType", "type": "string"}
        ],
        "name": "storeMemory",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [{"internalType": "string", "name": "_memoryId", "type": "string"}],
        "name": "retrieveMemory",
        "outputs": [{"internalType": "string", "name": "", "type": "string"}],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [{"internalType": "string", "name": "_memoryId", "type": "string"}],
        "name": "getMemoryDetails",
        "outputs": [
            {"internalType": "string", "name": "ipfsCID", "type": "string"},
            {"internalType": "address", "name": "owner", "type": "address"},
            {"internalType": "uint256", "name": "timestamp", "type": "uint256"},
            {"internalType": "string", "name": "agentId", "type": "string"},
            {"internalType": "string", "name": "taskType", "type": "string"}
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "getMemoryCount",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# Create contract instance
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=CONTRACT_ABI)

# Create account from private key
account = w3.eth.account.from_key(PRIVATE_KEY)
print(f"\n🔐 Using wallet: {account.address}")
print(f"💰 Balance: {w3.from_wei(w3.eth.get_balance(account.address), 'ether'):.6f} ETH")

# Check contract
memory_count = contract.functions.getMemoryCount().call()
print(f"\n✅ Contract connected!")
print(f"📦 Total memories stored: {memory_count}")

In [ ]:
# Cell 6: Store Memory on Blockchain
import time

def store_memory_on_chain(memory_id, ipfs_cid, agent_id, task_type):
    """Store IPFS hash on blockchain"""
    
    print(f"\n📤 Preparing transaction...")
    
    nonce = w3.eth.get_transaction_count(account.address)
    
    txn = contract.functions.storeMemory(
        memory_id,
        ipfs_cid,
        agent_id,
        task_type
    ).build_transaction({
        'from': account.address,
        'nonce': nonce,
        'gas': 300000,
        'gasPrice': w3.eth.gas_price,
        'chainId': 11155111  # Sepolia
    })
    
    signed_txn = account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    
    print(f"📤 Transaction sent: 0x{tx_hash.hex()}")
    print(f"🔗 View on Etherscan: https://sepolia.etherscan.io/tx/0x{tx_hash.hex()}")
    
    print("⏳ Waiting for confirmation (10-20 seconds)...")
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash, timeout=120)
    
    if receipt['status'] == 1:
        print(f"✅ SUCCESS! Memory stored on blockchain!")
        print(f"⛽ Gas used: {receipt['gasUsed']}")
        print(f"📦 Block: {receipt['blockNumber']}")
        return tx_hash.hex()
    else:
        print("❌ Transaction failed")
        return None

# Example: Store a memory
memory_id = f"UAV001_{int(time.time())}"
ipfs_cid = ipfs_hash  # From Cell 4

print(f"🚀 Storing memory '{memory_id}' on blockchain...")
print(f"📦 IPFS CID: {ipfs_cid}")

tx_hash = store_memory_on_chain(
    memory_id,
    ipfs_cid,
    "UAV-001",
    "obstacle_navigation"
)

if tx_hash:
    print(f"\n🎉 DAMN IS WORKING! Memory stored on both IPFS and Blockchain!")

In [ ]:
# Cell 7: Complete DAMN Agent Class
from datetime import datetime
import time

class DAMNAgent:
    """Decentralized AI Memory Network Agent"""
    
    def __init__(self, agent_id, contract, w3, account, pinata):
        self.agent_id = agent_id
        self.contract = contract
        self.w3 = w3
        self.account = account
        self.pinata = pinata
        self.memories_created = 0
        self.memories_accessed = 0
    
    def store_experience(self, task_type, data):
        """Store a new experience to IPFS + Blockchain"""
        
        memory = {
            "agent_id": self.agent_id,
            "task": task_type,
            "data": data,
            "timestamp": datetime.now().isoformat(),
            "created_at_block": self.w3.eth.block_number
        }
        
        print(f"📤 [{self.agent_id}] Uploading experience to IPFS...")
        result = self.pinata.pin_json(memory, f"{self.agent_id}_{task_type}")
        ipfs_cid = result['IpfsHash']
        print(f"✅ IPFS CID: {ipfs_cid}")
        
        memory_id = f"{self.agent_id}_{int(time.time())}"
        
        print(f"📝 [{self.agent_id}] Writing to blockchain...")
        nonce = self.w3.eth.get_transaction_count(self.account.address)
        
        txn = self.contract.functions.storeMemory(
            memory_id,
            ipfs_cid,
            self.agent_id,
            task_type
        ).build_transaction({
            'from': self.account.address,
            'nonce': nonce,
            'gas': 300000,
            'gasPrice': self.w3.eth.gas_price,
            'chainId': 11155111
        })
        
        signed_txn = self.account.sign_transaction(txn)
        tx_hash = self.w3.eth.send_raw_transaction(signed_txn.raw_transaction)
        
        print(f"⏳ [{self.agent_id}] Waiting for confirmation...")
        receipt = self.w3.eth.wait_for_transaction_receipt(tx_hash, timeout=120)
        
        if receipt['status'] == 1:
            print(f"✅ [{self.agent_id}] Experience stored successfully!")
            self.memories_created += 1
            return memory_id, ipfs_cid
        else:
            print(f"❌ [{self.agent_id}] Transaction failed")
            return None, None
    
    def retrieve_experience(self, memory_id):
        """Retrieve experience from blockchain → IPFS"""
        
        print(f"\n🔍 [{self.agent_id}] Retrieving memory: {memory_id}")
        
        try:
            ipfs_cid = self.contract.functions.retrieveMemory(memory_id).call()
            print(f"📦 Found CID on blockchain: {ipfs_cid}")
        except Exception as e:
            print(f"❌ Memory not found: {e}")
            return None
        
        try:
            memory = self.pinata.get_json(ipfs_cid)
            print(f"✅ [{self.agent_id}] Retrieved: {memory['task']} by {memory['agent_id']}")
            self.memories_accessed += 1
            return memory
        except Exception as e:
            print(f"❌ Failed to retrieve from IPFS: {e}")
            return None
    
    def get_stats(self):
        return {
            "agent_id": self.agent_id,
            "memories_created": self.memories_created,
            "memories_accessed": self.memories_accessed
        }

print("✅ DAMN Agent class created!")

In [ ]:
# Cell 8: Multi-Agent Demonstration
print("="*60)
print("🤖 DAMN NETWORK DEMONSTRATION")
print("="*60)

# Create two agents
uav1 = DAMNAgent("UAV-001", contract, w3, account, pinata)
uav2 = DAMNAgent("UAV-002", contract, w3, account, pinata)

print("\n🚁 UAV-001 encounters obstacle and learns...")
print("-"*60)

# UAV-1 experiences an obstacle
obstacle_experience = {
    "location": {"lat": 28.6139, "lon": 77.2090, "alt": 150},
    "obstacle_type": "building",
    "obstacle_height_m": 45,
    "safe_maneuver": "climb_to_200m_then_proceed",
    "success_rate": 0.98,
    "weather": "clear",
    "wind_speed_kmh": 12
}

mem_id_1, cid_1 = uav1.store_experience("obstacle_avoidance", obstacle_experience)

print("\n" + "="*60)
print("🚁 UAV-002 approaching same area...")
print("-"*60)

# UAV-2 checks network for relevant memories
time.sleep(2)

print(f"\n🔍 UAV-002 checking network for obstacle data...")
retrieved_memory = uav2.retrieve_experience(mem_id_1)

if retrieved_memory:
    print("\n💡 UAV-002 applying learned experience:")
    print(f"   → Location: {retrieved_memory['data']['location']}")
    print(f"   → Obstacle: {retrieved_memory['data']['obstacle_type']}")
    print(f"   → Safe maneuver: {retrieved_memory['data']['safe_maneuver']}")
    print(f"   → Success rate: {retrieved_memory['data']['success_rate']*100}%")
    print("\n✅ UAV-002 successfully navigated using UAV-001's experience!")

# Network statistics
print("\n" + "="*60)
print("📊 DAMN NETWORK STATISTICS")
print("-"*60)

total_memories = contract.functions.getMemoryCount().call()
uav1_stats = uav1.get_stats()
uav2_stats = uav2.get_stats()

print(f"\nUAV-001: {uav1_stats['memories_created']} created, {uav1_stats['memories_accessed']} accessed")
print(f"UAV-002: {uav2_stats['memories_created']} created, {uav2_stats['memories_accessed']} accessed")
print(f"\n🌐 Total network memories: {total_memories}")

print("\n" + "="*60)
print("🎉 DAMN DEMONSTRATION COMPLETE!")
print("="*60)